In [1]:
import pandas as pd
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from gensim.models import KeyedVectors
import re
import gensim

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aryuska/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
df = pd.read_csv("./description-final.csv")


In [3]:
df=df.dropna()

In [4]:

def _removeNonAscii(s):
    return "".join(i.encode('ascii', 'ignore').decode() for i in s )

def make_lower_case(text):
    return text.lower()

def remove_stop_words(text):
    text = text.split()
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    return text

def remove_html(text):
    html_pattern = re.compile('<.*?>')
    return html_pattern.sub(r'', text)

def remove_punctuation(text):
    tokenizer = RegexpTokenizer(r'\w+')
    text = tokenizer.tokenize(text)
    text = " ".join(text)
    return text

In [5]:
df['cleaned'] = df['description'].apply(_removeNonAscii)

df['cleaned'] = df.cleaned.apply(func = make_lower_case)
df['cleaned'] = df.cleaned.apply(func = remove_stop_words)
df['cleaned'] = df.cleaned.apply(func=remove_punctuation)
df['cleaned'] = df.cleaned.apply(func=remove_html)

In [6]:
corpus = []
for words in df['cleaned']:
    corpus.append(words.split())

In [7]:
model = gensim.models.Word2Vec(corpus, min_count=1)

In [8]:
df_rat = pd.read_csv("./ml-25m/ratings.csv")

In [9]:
df_rat_mov = pd.merge(df_rat, df, on="movieId", how="inner")

In [21]:
df_rat_mov_3 = df_rat_mov[df_rat_mov['userId'] == 3]
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
0,3,101864,4.0,1439474698,Oblivion,A veteran assigned to extract Earth's remainin...,veteran assigned extract earth s remaining res...
5339,3,102125,4.0,1439474937,Iron Man Three,When Tony Stark's world is torn apart by a for...,tony stark s world torn apart formidable terro...
13604,3,102407,4.0,1566090339,The Great Gatsby,"A writer and wall street trader, Nick, finds h...",writer wall street trader nick finds drawn pas...
18250,3,102445,4.0,1439473797,Star Trek Into Darkness,After the crew of the Enterprise find an unsto...,crew enterprise find unstoppable force terror ...
25713,3,102686,3.0,1566091223,The Hangover Part III,When one of their own is kidnapped by an angry...,one kidnapped angry gangster wolf pack must tr...
...,...,...,...,...,...,...,...
581873,3,176371,4.5,1566090291,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,young blade runner k s discovery long buried s...
586992,3,179819,3.5,1566090591,Star Wars: Episode VIII - The Last Jedi,The Star Wars saga continues as new heroes and...,star wars saga continues new heroes galactic l...
590299,3,182823,3.0,1566089510,Bright,A detective must work with an Orc to find a po...,detective must work orc find powerful wand evi...
591226,3,187541,3.5,1566090693,Incredibles 2,The Incredibles family takes on a new mission ...,incredibles family takes new mission involves ...


In [24]:
df_rat_mov_3 = df_rat_mov_3.sort_values(by='rating', ascending=False)
df_rat_mov_3 = df_rat_mov_3[df_rat_mov_3['rating'] >= 4.5]

In [25]:
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
507784,3,136449,5.0,1484753762,Kôkaku kidôtai 2.0,A hacker known as the Puppet Master is hunted ...,hacker known puppet master hunted female cybor...
253540,3,112552,5.0,1439472513,Whiplash,A promising young drummer enrolls at a cut-thr...,promising young drummer enrolls cut throat mus...
264719,3,112556,5.0,1439473517,Gone Girl,With his wife's disappearance having become th...,wife s disappearance become focus intense medi...
327627,3,115713,4.5,1439474672,Ex Machina,A young programmer is selected to participate ...,young programmer selected participate ground b...
581873,3,176371,4.5,1566090291,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,young blade runner k s discovery long buried s...
118851,3,106782,4.5,1439473659,The Wolf of Wall Street,"Based on the true story of Jordan Belfort, fro...",based true story jordan belfort rise wealthy s...
594096,3,187593,4.5,1566090414,Deadpool 2,Foul-mouthed mutant mercenary Wade Wilson (a.k...,foul mouthed mutant mercenary wade wilson a k ...
68805,3,104243,4.5,1439474932,Riddick,"Left for dead on a sun-scorched planet, Riddic...",left dead sun scorched planet riddick finds al...


In [71]:
user_pref = ""
for i,data in df_rat_mov_3.iterrows():
    user_pref += " " + data['cleaned']

' hacker known puppet master hunted female cyborg cop partner film revised version kkaku kidtai 1995 wife s disappearance become focus intense media circus man sees spotlight turned suspected may innocent promising young drummer enrolls cut throat music conservatory dreams greatness mentored instructor stop nothing realize student s potential young programmer selected participate ground breaking experiment synthetic intelligence evaluating human qualities highly advanced humanoid a i left dead sun scorched planet riddick finds alien race predators activating emergency beacon alerts two ships one carrying new breed mercenary captained man riddick s past young blade runner k s discovery long buried secret leads track former blade runner rick deckard who s missing thirty years based true story jordan belfort rise wealthy stock broker living high life fall involving crime corruption federal government foul mouthed mutant mercenary wade wilson a k a deadpool assembles team fellow mutant rog

In [79]:
corpus_user_pref = []
corpus_user_pref.append(user_pref.split())
    

In [80]:
corpus_user_pref

[['hacker',
  'known',
  'puppet',
  'master',
  'hunted',
  'female',
  'cyborg',
  'cop',
  'partner',
  'film',
  'revised',
  'version',
  'kkaku',
  'kidtai',
  '1995',
  'wife',
  's',
  'disappearance',
  'become',
  'focus',
  'intense',
  'media',
  'circus',
  'man',
  'sees',
  'spotlight',
  'turned',
  'suspected',
  'may',
  'innocent',
  'promising',
  'young',
  'drummer',
  'enrolls',
  'cut',
  'throat',
  'music',
  'conservatory',
  'dreams',
  'greatness',
  'mentored',
  'instructor',
  'stop',
  'nothing',
  'realize',
  'student',
  's',
  'potential',
  'young',
  'programmer',
  'selected',
  'participate',
  'ground',
  'breaking',
  'experiment',
  'synthetic',
  'intelligence',
  'evaluating',
  'human',
  'qualities',
  'highly',
  'advanced',
  'humanoid',
  'a',
  'i',
  'left',
  'dead',
  'sun',
  'scorched',
  'planet',
  'riddick',
  'finds',
  'alien',
  'race',
  'predators',
  'activating',
  'emergency',
  'beacon',
  'alerts',
  'two',
  'ships'

In [91]:
similar_words = model.wv.n_similarity(corpus_user_pref[0],['antihero'])
similar_words

0.0

In [118]:
google_model.train(corpus, total_examples=google_model.corpus_count, epochs = 5)

(4921140, 5164280)

In [130]:
print(model.wv.n_similarity(corpus_user_pref[0], corpus[0]))

0.9044568


In [122]:
print(google_model.wv.similar_by_word(topn=10,word='soldier'))

[('veteran', 0.9350651502609253), ('cluster', 0.9003807306289673), ('indian', 0.8781213760375977), ('vet', 0.8761188387870789), ('deserter', 0.8735191226005554), ('paratrooper', 0.8707262873649597), ('kurdish', 0.8644377589225769), ('vietnam', 0.8643453121185303), ('pow', 0.8641802668571472), ('chechen', 0.8607069253921509)]


In [125]:
print(model.wv.similar_by_word(topn=10,word='soldier'))

[('veteran', 0.9518615007400513), ('vet', 0.9358885288238525), ('prisoner', 0.9101263880729675), ('chechen', 0.906245231628418), ('correspondent', 0.9054422378540039), ('refugee', 0.8952005505561829), ('vietnam', 0.8936744332313538), ('beret', 0.8912519216537476), ('indian', 0.8816993832588196), ('onset', 0.879857063293457)]


In [129]:
df_recommend = df
df_recommend

,title,description,movieId,cleaned
0,Waiting for Lightning,A documentary on pro skateboarder Danny Way's ...,99513,documentary pro skateboarder danny way s tough...
1,Yesterday's Enemy,"In World War II, during the Japanese invasion ...",99515,world war ii japanese invasion burma lost remn...
2,Young People,A show-business family leaves the Great White ...,99517,show business family leaves great white way he...
3,Henry,Henry is a cynical man who owns a musical inst...,99519,henry cynical man owns musical instruments sto...
4,L'incruste,A small crook manages to crash at a musician's...,99521,small crook manages crash musician s apartment...
...,...,...,...,...
43322,Wij,Eight teens - four guys and four girls - are t...,209157,eight teens four guys four girls best friends ...
43323,Janela da Alma,Nineteen people with differing degrees of visu...,209159,nineteen people differing degrees visual impai...
43324,Rossz versek,"33-years old Tamás Merthner is heartbroken, af...",209163,33 years old tams merthner heartbroken girlfri...
43325,A Girl Thing,An NYC shrink sees four different women with d...,209169,nyc shrink sees four different women different...


In [133]:
def get_similarity_scoew(text):
    return model.wv.n_similarity(corpus_user_pref[0], text.split())

In [134]:
df_recommend['score'] = df.cleaned.apply(get_similarity_scoew)

/var/folders/wp/36kdrjm51z92ythp0vp8xp5m0000gn/T/ipykernel_77752/925198670.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_recommend['score'] = df.cleaned.apply(get_similarity_scoew)


In [136]:
df_recommend = df_recommend.sort_values(by='score', ascending=False)

In [141]:
df_rat_mov_3

,userId,movieId,rating,timestamp,title,description,cleaned
507784,3,136449,5.0,1484753762,Kôkaku kidôtai 2.0,A hacker known as the Puppet Master is hunted ...,hacker known puppet master hunted female cybor...
264719,3,112556,5.0,1439473517,Gone Girl,With his wife's disappearance having become th...,wife s disappearance become focus intense medi...
253540,3,112552,5.0,1439472513,Whiplash,A promising young drummer enrolls at a cut-thr...,promising young drummer enrolls cut throat mus...
327627,3,115713,4.5,1439474672,Ex Machina,A young programmer is selected to participate ...,young programmer selected participate ground b...
68805,3,104243,4.5,1439474932,Riddick,"Left for dead on a sun-scorched planet, Riddic...",left dead sun scorched planet riddick finds al...
581873,3,176371,4.5,1566090291,Blade Runner 2049,Young Blade Runner K's discovery of a long-bur...,young blade runner k s discovery long buried s...
118851,3,106782,4.5,1439473659,The Wolf of Wall Street,"Based on the true story of Jordan Belfort, fro...",based true story jordan belfort rise wealthy s...
594096,3,187593,4.5,1566090414,Deadpool 2,Foul-mouthed mutant mercenary Wade Wilson (a.k...,foul mouthed mutant mercenary wade wilson a k ...


In [151]:
df_recommend = df_recommend[:10]

In [155]:
df_recommend

,title,description,movieId,cleaned,score
32963,Who Is Clark Rockefeller?,"Called ""the longest running con in FBI history...",184487,called the longest running con fbi history chr...,0.992058
4389,Fortune and Men's Eyes,"The young, naive Smitty is sent to prison for ...",118232,young naive smitty sent prison six months cath...,0.991181
25465,Les Patterson Saves the World,Fat middle aged 24/7 drunkard Les Patterson (B...,168482,fat middle aged 24 7 drunkard les patterson ba...,0.991004
33231,Le plaisir de chanter,A banker has died. He trafficked in nuclear ma...,185043,banker died trafficked nuclear material french...,0.990824
16531,'Z' fung bou,This is a story about the biggest financial fr...,148270,story biggest financial fraud attempted hong k...,0.990312
39591,Soldados de Salamina,When the professor and writer Lola Sánchez is ...,199131,professor writer lola snchez assigned write co...,0.989737
13657,2033,"The first installment of a planned trilogy, di...",141652,first installment planned trilogy director fra...,0.989707
5406,Sons o' Guns,Broadway star Jimmy Canfield stars in a patrio...,121642,broadway star jimmy canfield stars patriotic s...,0.989305
7479,Kosmicheskiy reys: Fantasticheskaya novella,"In 1946, the Soviet space program is undergoin...",126436,1946 soviet space program undergoing turmoil p...,0.989061
17322,Proof of the Devil,A string of murders ended in the capture of an...,150190,string murders ended capture 18 yo boy named j...,0.988768
